# Adding interactive controls

We will be adding interactive controls to our graphs using the ```ipywidgets``` [library](https://ipywidgets.readthedocs.io/en/stable/index.html). This is basically a Graphical User Interface ([GUI](https://en.wikipedia.org/wiki/Graphical_user_interface)) library that runs in a notebook. Its control elements (so-called [widgets](https://en.wikipedia.org/wiki/Graphical_widget)) are actually an interface to [JavaScript](https://en.wikipedia.org/wiki/JavaScript) code that runs in your browser (in a stardard GUI library meant for desktop applications, they would be written in some low-level language and interface with the [window manager](https://en.wikipedia.org/wiki/Window_manager) on your computer). An in-depth treatment of GUIs is beyond the scope of this tutorial. However, I will strive to highlight concepts of general applicability and to use a clean syntax that highlights where the various elements fit in and may be easier to port to other GUI toolkits. If you are interested in learning more about (desktop-based) GUIs in Python, I strongly recommend the [appJar](http://appjar.info/) library - it is an excellent learning library and a great prototyping tool as well.

The ```ipywidgets``` library is an integral part of the Jupyter framework and should therefore already be installed on any machine you are running this notebook on. If for some reason it isn't, follow these [instructions](https://ipywidgets.readthedocs.io/en/stable/user_install.html) to install it.

In [1]:
from IPython.display import clear_output
import ipywidgets as wdg
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
%matplotlib inline
# make figures larger
plt.rcParams['figure.dpi'] = 100

## Updating your data

It is a good idea to provide some data on the disk for your dashboard to start on - after all, the PHE website might be offline when someone accesses your dashboard; also, accessing the PHE website directly would make startup time longer. However, you can provide a button that the user can click to poll the API and refresh the data.

The ```ipywidgets``` library provides a ```Button``` class that implements a [clickable button](https://ipywidgets.readthedocs.io/en/latest/examples/Widget%20List.html?highlight=button#Button). The way this is linked to your code is through a [callback function](https://en.wikipedia.org/wiki/Callback_(computer_programming)). This is a function that is passed as a parameter to the ```on_click``` method of the ```Button``` object, that in turn calls it when it is clicked. The callback function is passed the ```Button``` object itself as a parameter (which is useful if more than one button, for instance, shares the same callback). We can safely ignore that here.

The call to ```display``` below actually draws the button on the notebook.

In [3]:
# our API access function. This will be called by the button when it is clicked
def access_api(button):
    # Ignore the button parameter
    #
    # put code for polling the API here
    #
    # after all is done, you can switch the icon on the button to a "check" sign
    # and optionally disable the button - it won't be needed again.
    apibutton.icon="check"
    apibutton.disabled=True

# see the doc for the parameters    
apibutton=wdg.Button(
    description='Refresh data',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Click to download current Public Health England data',
    icon='download' # (FontAwesome names without the `fa-` prefix)
)

# register the callback function with the button
apibutton.on_click(access_api)

# this is an iPython function that generalises print for Jupyter Notebooks; we use it to 
# display the widgets
display(apibutton)

Button(description='Refresh data', icon='download', style=ButtonStyle(), tooltip='Click to download current Pu…

## Single-control graphs

Adding an interactive control to a graph is not much more complicated. The control can be any of the many [widgets](https://ipywidgets.readthedocs.io/en/latest/examples/Widget%20List.html#) provided by ```iPywidgets```. Here we have chosen ```SelectMultiple``` that allows us to select the DataFrame columns to be plotted in the Age Distribution graphs - namely, any combination of the *males*, *females* or *total* columns (see the [doc](https://ipywidgets.readthedocs.io/en/latest/examples/Widget%20List.html#SelectMultiple)).

Again, ```SelectMultiple``` makes use of a callback, that gets passed the list of options that have been selected. Each time the selection is changed, the callback function is called again. The way the callback is registered with the widget is slightly different: the callback (here ```age_graph```) must take a list parameter (the list of columns). We then pass the callback to ```interactive_output```, together with a dictionary that specifies which widget (```agecols```) must provide a value for the callback parameter (```graphcolumns```). Basically, the ```iPywidget``` framework  calls ```age_graph``` with the values retrieved from the widget after each user interaction.  The value of ```interactive_output``` is an ```Output``` widget (see the [doc](https://ipywidgets.readthedocs.io/en/latest/examples/Output%20Widget.html)), that can be assigned to a variable and passed to ```display``` together with the ```agecols``` widget itself. Any printout, graph or other output of the callback ```age_graph``` is captured and displayed through that ```Output``` widget, that renders it in the browser.

In [29]:
# in the dashboard, this will be the dataframe that's computed directly from the json files,
# or from the current data downloaded from the API
age_df=pd.read_pickle("agedf.pkl")

In [30]:
agecols=wdg.SelectMultiple(
    options=['males', 'females', 'total'], # options available
    value=['males', 'females'], # initial value
    rows=3, # rows of the selection box
    description='Sex',
    disabled=False
)

def age_graph(graphcolumns):
    # our callback function.
    ncols=len(graphcolumns)
    if ncols>0:
        age_df.plot(kind='bar', y=list(graphcolumns)) # graphcolumns is a tuple - we need a list
        # plt.show() # important - graphs won't update properly if this is missing
    else:
        # if the user has not selected any column, print a message instead
        print("Click to select data for graph")
        print("(CTRL-Click to select more than one category)")
    
# keep calling age_graph(graphcolumns=value_of_agecols); capture output in widget output    
output=wdg.interactive_output(age_graph, {'graphcolumns': agecols})

display(agecols, output)

SelectMultiple(description='Sex', index=(0, 1), options=('males', 'females', 'total'), rows=3, value=('males',…

Output()

## Multiple-control graphs

In the time series graph, we may want to add a couple of controls - one for selecting the stats to display, and the other to choose between an linear and logarithmic scale. This second choice we can implement with a ```RadioButtons``` widget ([documentation](https://ipywidgets.readthedocs.io/en/latest/examples/Widget%20List.html#RadioButtons)). This widget returns a Boolean value (```True``` or ```False```) to the callback function. The callback (in our case ```timeseries_graph```) must then accept two parameteres,  a tuple of stats coming from the ```SelectMultiple``` widget and the Boolean value from the ```RadioButtons```. Again, the ```interactive_output``` function associates the widgets with the callback and captures the output in an ```Output``` widget.

The new element in this example is the [container widget](https://ipywidgets.readthedocs.io/en/latest/examples/Widget%20List.html#Container/Layout-widgets) ```HBox```. Container widgets are another feature shared by most GUI toolkits; they are used to group  and arrange other widgets on the interface.

In [7]:
# in the dashboard, this should be computed from the json files
timeseriesdf=pd.read_pickle("timeseriesdf.pkl")

In [ ]:
series=wdg.SelectMultiple(
    options=['cases', 'hospital', 'deaths'],
    value=['cases', 'hospital', 'deaths'],
    rows=3,
    description='Stats:',
    disabled=False
)

scale=wdg.RadioButtons(
    options=['linear', 'log'],
#    value='pineapple', # Defaults to 'pineapple'
#    layout={'width': 'max-content'}, # If the items' names are long
    description='Scale:',
    disabled=False
)

# try replacing HBox with a VBox
controls=wdg.HBox([series, scale])

def timeseries_graph(gcols, gscale):
    if gscale=='linear':
        logscale=False
    else:
        logscale=True
    ncols=len(gcols)
    if ncols>0:
        timeseriesdf[list(gcols)].plot(logy=logscale)
        plt.show() # important - graphs won't update if this is missing 
    else:
        print("Click to select data for graph")
        print("(CTRL-Click to select more than one category)")

# keep calling timeseries_graph(gcols=value_of_series, gscale=value_of_scale); 
# capture output in widget graph   
graph=wdg.interactive_output(timeseries_graph, {'gcols': series, 'gscale': scale})

display(controls, graph)

Note how the ```HBox``` widget allows us to display the two control widgets side by side. Containers can be nested, so for instance an ```HBox``` can contain a ```VBox```. Since ```graph``` is also a widget (in fact, an ```Output``` widget), this allows us to be more adventurous and explore more advanced layouts:

In [ ]:
# stack series and scale on top of each other
ctrls=wdg.VBox([series, scale])
# put the graph and the controls side by side
form=wdg.HBox([graph, ctrls])

# Now form contains the entire interface
display(form)

Note that since, in this case, we are re-using the same widgets (that are simply re-drawn in a different layout), the two graphs and the controls are exact copies of each other and cannot be modified independently - changing one copy will change the other. The detailed [layout of widgets](https://ipywidgets.readthedocs.io/en/latest/examples/Widget%20Styling.html) is an intricate topic that goes beyond the scope of this tutorial. 

## Your turn

Load the *pickle* file with the ```DataFrame``` you saved at the end of the previous notebook and add interactive controls to your graph. You can pick one of more widgets from this [list](https://ipywidgets.readthedocs.io/en/latest/examples/Widget%20List.html), including any containers if required. I recommend that, at least initially, you stick with what is simple and just works.

In [8]:
from IPython.display import clear_output
import ipywidgets as wdg
import pandas as pd
import matplotlib.pyplot as plt

%matplotlib inline
plt.rcParams['figure.dpi'] = 100

age_distribution_df = pd.read_pickle('age_distribution_df.pkl')
case_trend_df = pd.read_pickle('case_trend_df.pkl')
vaccination_trend_df = pd.read_pickle('vaccination_trend_df.pkl')

age_cols = wdg.SelectMultiple(
    options = ['males', 'females', 'total'],
    value = ['males', 'females'],
    rows = 3,
    description = 'Gender',
    disabled = False
)
def age_distribution_graph(graph_cols):
    n_cols = len(graph_cols)
    if n_cols > 0:
        age_distribution_df.plot(kind = 'bar', y = list(graph_cols))
        plt.show()
    else:
        print("Click to select data for graph")
        print("(CTRL-Click to select more than one category)")
age_distribution_output = wdg.interactive_output(
    age_distribution_graph,
    {'graph_cols': age_cols}
)
age_controls = wdg.VBox([age_cols])
age_form = wdg.HBox([age_distribution_output, age_controls])
display(age_form)

case_cols = wdg.SelectMultiple(
    options = ['daily_cases', 'daily_admissions', 'daily_deaths'],
    value = ['daily_cases', 'daily_admissions', 'daily_deaths'],
    rows = 3,
    description = 'Stats:',
    disabled = False
)
case_scale = wdg.RadioButtons(
    options = ['linear', 'log'],
    description = 'Scale:',
    disabled = False
)
def case_graph(graph_cols, graph_scale):
    if graph_scale == 'linear':
        log_scale = False
    else:
        log_scale = True
    n_cols = len(graph_cols)
    if n_cols > 0:
        case_trend_df[list(graph_cols)].plot(logy = log_scale)
        plt.show()
    else:
        print("Click to select data for graph")
        print("(CTRL-Click to select more than one category)")
case_trend_output = wdg.interactive_output(
    case_graph,
    {'graph_cols': case_cols, 'graph_scale': case_scale}
)
case_controls = wdg.VBox([case_cols, case_scale])
case_form = wdg.HBox([case_trend_output, case_controls])
display(case_form)

vaccination_cols = wdg.SelectMultiple(
    options = ['first_dose', 'second_dose', 'booster_dose'],
    value = ['first_dose', 'second_dose', 'booster_dose'],
    rows = 3,
    description = 'Stats:',
    disabled = False
)
vaccination_scale = wdg.RadioButtons(
    options = ['linear', 'log'],
    description = 'Scale:',
    disabled = False
)
def vaccination_graph(graph_cols, graph_scale):
    if graph_scale == 'linear':
        log_scale = False
    else:
        log_scale = True
    n_cols = len(graph_cols)
    if n_cols > 0:
        vaccination_trend_df[list(graph_cols)].plot(logy = log_scale)
        plt.show()
    else:
        print("Click to select data for graph")
        print("(CTRL-Click to select more than one category)")
vaccination_trend_output = wdg.interactive_output(
    vaccination_graph,
    {'graph_cols': vaccination_cols, 'graph_scale': vaccination_scale}
)
vaccination_controls = wdg.VBox([vaccination_cols, vaccination_scale])
vaccination_form = wdg.HBox([vaccination_trend_output, vaccination_controls])
display(vaccination_form)




**(C) 2020 Fabrizio Smeraldi** ([f.smeraldi@qmul.ac.uk](mailto:f.smeraldi@qmul.ac.uk) - [web](http://www.eecs.qmul.ac.uk/~fabri/)), all rights reserved.